In [ ]:
# import os
# import pyspark

# # Set environment variables
# os.environ['PYSPARK_PYTHON'] = 'python3.11'
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.11'
# import os
# os.environ["PYSPARK_PYTHON"] = "python3"

In [ ]:
# !pip install pyodbc pandas pyspark


In [24]:
# pip install pyodbc pandas
import pyodbc
import pandas as pd
import pyspark
import random
from pyspark.sql import SparkSession

In [25]:
# pip install "pymongo[srv]"

In [26]:
from pyspark.sql import SparkSession

# Create Spark session
spark = SparkSession.builder \
    .appName("MySparkApp") \
    .config("spark.driver.memory","4g")\
    .config("spark.sql.legacy.timeParserPolicy","LEGACY")\
    .getOrCreate()


In [27]:
import pyodbc
import pandas as pd

# Connection string with raw string notation
conn = pyodbc.connect(r'Driver={SQL Server};'
                      r'Server=BUMBLEBEE;'
                      r'Database=ShootingIncidentNYPD;'
                      r'Trusted_Connection=yes;')

# Create a cursor object
cursor = conn.cursor()

# Define your query
query1 = 'SELECT [INCIDENT_KEY],[OCCUR_DATE],[OCCUR_TIME],[BORO] ,[LOC_OF_OCCUR_DESC],[PRECINCT],[JURISDICTION_CODE],[LOC_CLASSFCTN_DESC],[LOCATION_DESC],[STATISTICAL_MURDER_FLAG],[PERP_AGE_GROUP],[PERP_SEX],[PERP_RACE],[VIC_AGE_GROUP],[VIC_SEX],[VIC_RACE],[X_COORD_CD],[Y_COORD_CD],[Latitude],[Longitude],[Lon_Lat] FROM [ShootingIncidentNYPD].[dbo].[NYPD_Shooting_Incident_Data__Historic_]'  
query2 = 'SELECT [INCIDENT_KEY] ,[OCCUR_DATE]      ,[OCCUR_TIME]      ,[BORO]      ,[LOC_OF_OCCUR_DESC]      ,[PRECINCT]      ,[JURISDICTION_CODE]      ,[LOC_CLASSFCTN_DESC]      ,[LOCATION_DESC]      ,[STATISTICAL_MURDER_FLAG]      ,[PERP_AGE_GROUP]      ,[PERP_SEX]      ,[PERP_RACE]      ,[VIC_AGE_GROUP]      ,[VIC_SEX]      ,[VIC_RACE]      ,[X_COORD_CD]      ,[Y_COORD_CD]      ,[Latitude]      ,[Longitude]      ,[New Georeferenced Column]  FROM [ShootingIncidentNYPD].[dbo].[NYPD_Shooting_Incident_Data__Year_To_Date_]'  

# Execute the query and fetch the data into a pandas DataFrame
df1 = pd.read_sql(query1, conn)
df2 = pd.read_sql(query2, conn)

# Display the DataFrame
print(df1.head())
print(df1.shape)

C:\Users\macwa\AppData\Local\Temp\ipykernel_13668\2936498768.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql(query1, conn)


  INCIDENT_KEY  OCCUR_DATE OCCUR_TIME       BORO LOC_OF_OCCUR_DESC PRECINCT  \
0     80086639  08/01/2011   21:41:00      BRONX                         40   
1    231941260  08/07/2021   20:49:00  MANHATTAN                         34   
2     73408255  06/26/2010   02:10:00      BRONX                         47   
3     32320695  06/16/2007   02:52:00     QUEENS                        103   
4    248661285  07/27/2022   19:45:00   BROOKLYN           OUTSIDE       75   

  JURISDICTION_CODE LOC_CLASSFCTN_DESC              LOCATION_DESC  \
0                 0                     MULTI DWELL - PUBLIC HOUS   
1                 0                                                 
2                 0                                                 
3                 0                                                 
4                 0             STREET                     (null)   

  STATISTICAL_MURDER_FLAG  ... PERP_SEX PERP_RACE VIC_AGE_GROUP VIC_SEX  \
0                    true  ...     

C:\Users\macwa\AppData\Local\Temp\ipykernel_13668\2936498768.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(query2, conn)


In [28]:
print(df2.head())
print(df2.shape)

  INCIDENT_KEY  OCCUR_DATE OCCUR_TIME      BORO LOC_OF_OCCUR_DESC PRECINCT  \
0    283390056  03/07/2024   17:15:00  BROOKLYN           OUTSIDE       73   
1    283404608  03/07/2024   16:14:00     BRONX           OUTSIDE       40   
2    281250471  01/27/2024   18:36:00  BROOKLYN           OUTSIDE       75   
3    283951187  03/18/2024   22:00:00    QUEENS           OUTSIDE      103   
4    279799858  01/01/2024   14:57:00     BRONX           OUTSIDE       52   

  JURISDICTION_CODE LOC_CLASSFCTN_DESC LOCATION_DESC STATISTICAL_MURDER_FLAG  \
0                 0             STREET        (null)                       N   
1                 0             STREET        (null)                       Y   
2                 0             STREET        (null)                       N   
3                 0           DWELLING     PVT HOUSE                       N   
4                 0             STREET        (null)                       N   

   ... PERP_SEX       PERP_RACE VIC_AGE_GROUP VIC_

In [29]:
# Merge DataFrames on 'INCIDENT_KEY'
df_combined = pd.concat([df1, df2], ignore_index=True)
df_combined.reset_index(inplace=True)

# Rename columns for better readability if needed
df_combined.rename(columns={'OCCUR_DATE_historic': 'Occurrence Date Historic',
                            'OCCUR_DATE_current': 'Occurrence Date Current',
                            'OCCUR_TIME_historic': 'Occurrence Time Historic',
                            'OCCUR_TIME_current': 'Occurrence Time Current',
                            'BORO_historic': 'Borough Historic',
                            'BORO_current': 'Borough Current'}, inplace=True)

df_combined = df_combined.fillna(method='ffill').fillna(method='bfill')

# Display the cleaned DataFrame
print(df_combined.head())
print(df_combined.shape)

C:\Users\macwa\AppData\Local\Temp\ipykernel_13668\3152273496.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_combined = df_combined.fillna(method='ffill').fillna(method='bfill')


   index INCIDENT_KEY  OCCUR_DATE OCCUR_TIME       BORO LOC_OF_OCCUR_DESC  \
0      0     80086639  08/01/2011   21:41:00      BRONX                     
1      1    231941260  08/07/2021   20:49:00  MANHATTAN                     
2      2     73408255  06/26/2010   02:10:00      BRONX                     
3      3     32320695  06/16/2007   02:52:00     QUEENS                     
4      4    248661285  07/27/2022   19:45:00   BROOKLYN           OUTSIDE   

  PRECINCT JURISDICTION_CODE LOC_CLASSFCTN_DESC              LOCATION_DESC  \
0       40                 0                     MULTI DWELL - PUBLIC HOUS   
1       34                 0                                                 
2       47                 0                                                 
3      103                 0                                                 
4       75                 0             STREET                     (null)   

   ... PERP_RACE VIC_AGE_GROUP VIC_SEX        VIC_RACE  \
0  ...    

In [30]:
df_combined_spark= spark.createDataFrame(df_combined)

In [31]:
df_combined_spark.show()

+-----+------------+----------+----------+---------+-----------------+--------+-----------------+------------------+--------------------+-----------------------+--------------+--------+--------------+-------------+-------+--------------+--------------------+--------------------+------------------+-------------------+--------------------+------------------------+
|index|INCIDENT_KEY|OCCUR_DATE|OCCUR_TIME|     BORO|LOC_OF_OCCUR_DESC|PRECINCT|JURISDICTION_CODE|LOC_CLASSFCTN_DESC|       LOCATION_DESC|STATISTICAL_MURDER_FLAG|PERP_AGE_GROUP|PERP_SEX|     PERP_RACE|VIC_AGE_GROUP|VIC_SEX|      VIC_RACE|          X_COORD_CD|          Y_COORD_CD|          Latitude|          Longitude|             Lon_Lat|New Georeferenced Column|
+-----+------------+----------+----------+---------+-----------------+--------+-----------------+------------------+--------------------+-----------------------+--------------+--------+--------------+-------------+-------+--------------+--------------------+------------

In [32]:
# Show the schema to understand data types
df_combined_spark.printSchema()


root
 |-- index: long (nullable = true)
 |-- INCIDENT_KEY: string (nullable = true)
 |-- OCCUR_DATE: string (nullable = true)
 |-- OCCUR_TIME: string (nullable = true)
 |-- BORO: string (nullable = true)
 |-- LOC_OF_OCCUR_DESC: string (nullable = true)
 |-- PRECINCT: string (nullable = true)
 |-- JURISDICTION_CODE: string (nullable = true)
 |-- LOC_CLASSFCTN_DESC: string (nullable = true)
 |-- LOCATION_DESC: string (nullable = true)
 |-- STATISTICAL_MURDER_FLAG: string (nullable = true)
 |-- PERP_AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- VIC_AGE_GROUP: string (nullable = true)
 |-- VIC_SEX: string (nullable = true)
 |-- VIC_RACE: string (nullable = true)
 |-- X_COORD_CD: string (nullable = true)
 |-- Y_COORD_CD: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Lon_Lat: string (nullable = true)
 |-- New Georeferenced Column: string (nullable = tru

In [33]:
from pyspark.sql.functions import col
import functools

# Combine conditions for all columns with non-null values
non_null_conditions = [col(c).isNotNull() for c in df_combined_spark.columns]
condition = functools.reduce(lambda x, y: x & y, non_null_conditions)

# Filter DataFrame based on the combined condition
df_combined_spark = df_combined_spark.filter(condition)


In [34]:
# Fill missing values in specific columns
from pyspark.sql.functions import col, lit, when, count, mean, median, mode, first

# Fill categorical columns with mode or placeholder
categorical_columns = [
    'BORO', 'LOC_OF_OCCUR_DESC', 'JURISDICTION_CODE', 'LOC_CLASSFCTN_DESC', 'LOCATION_DESC',
    'PERP_AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'VIC_AGE_GROUP', 'VIC_SEX', 'VIC_RACE'
]

for column in categorical_columns:
    mode_value = df_combined_spark.select(column).groupBy(column).count().orderBy("count", ascending=False).first()[0]
    df_combined_spark = df_combined_spark.withColumn(column, when(col(column).isNull(), mode_value).otherwise(col(column)))

# Fill numerical columns 
numerical_columns = ['PRECINCT', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude']
for column in numerical_columns:
    mean_value = df_combined_spark.select(mean(col(column))).first()[0]
    df_combined_spark = df_combined_spark.withColumn(column, when(col(column).isNull(), mean_value).otherwise(col(column)))

# Fill date/time columns with placeholders
date_columns = ['OCCUR_DATE', 'OCCUR_TIME']
for column in date_columns:
    placeholder_value = '1900-01-01' if column == 'OCCUR_DATE' else '00:00:00'
    df_combined_spark = df_combined_spark.withColumn(column, when(col(column).isNull(), lit(placeholder_value)).otherwise(col(column)))


In [35]:
from pyspark.sql.functions import to_date, col

# Convert 'OCCUR_DATE' to DateType
df_combined_spark = df_combined_spark.withColumn('OCCUR_DATE', to_date(col('OCCUR_DATE'), 'MM/dd/yyyy'))

In [36]:
from pyspark.sql.functions import lower

# Convert categorical columns to lowercase
categorical_columns = ['BORO', 'LOC_OF_OCCUR_DESC', 'PERP_AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'VIC_AGE_GROUP', 'VIC_SEX', 'VIC_RACE']
for col_name in categorical_columns:
    df_combined_spark = df_combined_spark.withColumn(col_name, lower(col(col_name)))

In [37]:
# Remove duplicate rows
df_combined_spark = df_combined_spark.dropDuplicates()

In [38]:
from pyspark.sql.functions import year, month

# Extract year and month from 'OCCUR_DATE'
df_combined_spark = df_combined_spark.withColumn('YEAR', year(col('OCCUR_DATE')))
df_combined_spark = df_combined_spark.withColumn('MONTH', month(col('OCCUR_DATE')))

In [39]:
from pyspark.sql.functions import col, sum, when

# Count the total number of rows
total_rows = df_combined_spark.count()

# Calculate the number of missing values in each column
missing_counts = df_combined_spark.select([(sum(col(c).isNull().cast("int"))).alias(c) for c in df_combined_spark.columns])

# Calculate the percentage of missing values
missing_percentages = missing_counts.select([(col(c) / total_rows * 100).alias(f"{c}_missing_pct") for c in df_combined_spark.columns])

# Show the missing percentages
missing_percentages.show()


+-----------------+------------------------+----------------------+----------------------+----------------+-----------------------------+--------------------+-----------------------------+------------------------------+-------------------------+-----------------------------------+--------------------------+--------------------+---------------------+-------------------------+-------------------+--------------------+----------------------+----------------------+--------------------+---------------------+-------------------+------------------------------------+----------------+-----------------+
|index_missing_pct|INCIDENT_KEY_missing_pct|OCCUR_DATE_missing_pct|OCCUR_TIME_missing_pct|BORO_missing_pct|LOC_OF_OCCUR_DESC_missing_pct|PRECINCT_missing_pct|JURISDICTION_CODE_missing_pct|LOC_CLASSFCTN_DESC_missing_pct|LOCATION_DESC_missing_pct|STATISTICAL_MURDER_FLAG_missing_pct|PERP_AGE_GROUP_missing_pct|PERP_SEX_missing_pct|PERP_RACE_missing_pct|VIC_AGE_GROUP_missing_pct|VIC_SEX_missing_pct|VIC

In [40]:
# Drop columns with more than a threshold percentage of missing values
threshold = 0.5 * total_rows

# Get a list of columns to drop
columns_to_drop = [column for column in df_combined_spark.columns if df_combined_spark.filter(col(column).isNull()).count() > threshold]

# Drop the columns
df_combined_spark = df_combined_spark.drop(*columns_to_drop)


df_combined_spark = df_combined_spark.drop('index','New Georeferenced Column','Lon_Lat','LOC_OF_OCCUR_DESC','JURISDICTION_CODE','LOC_CLASSFCTN_DESC')

In [41]:
df_combined_spark.printSchema()

root
 |-- INCIDENT_KEY: string (nullable = true)
 |-- OCCUR_DATE: date (nullable = true)
 |-- OCCUR_TIME: string (nullable = true)
 |-- BORO: string (nullable = true)
 |-- PRECINCT: string (nullable = true)
 |-- LOCATION_DESC: string (nullable = true)
 |-- STATISTICAL_MURDER_FLAG: string (nullable = true)
 |-- PERP_AGE_GROUP: string (nullable = true)
 |-- PERP_SEX: string (nullable = true)
 |-- PERP_RACE: string (nullable = true)
 |-- VIC_AGE_GROUP: string (nullable = true)
 |-- VIC_SEX: string (nullable = true)
 |-- VIC_RACE: string (nullable = true)
 |-- X_COORD_CD: string (nullable = true)
 |-- Y_COORD_CD: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)



In [42]:
#Removing Outliers 
from pyspark.sql.functions import col

# Assuming "X_COORD_CD" and "Y_COORD_CD" are strings (cast if necessary)
df_combined_spark = df_combined_spark.withColumn("X_COORD_CD_numeric", col("X_COORD_CD").cast("double"))
df_combined_spark = df_combined_spark.withColumn("Y_COORD_CD_numeric", col("Y_COORD_CD").cast("double"))

# Removing outliers using IQR method for 'X_COORD_CD' (using the casted column)
Q1 = df_combined_spark.approxQuantile("X_COORD_CD_numeric", [0.25], 0.05)
Q3 = df_combined_spark.approxQuantile("X_COORD_CD_numeric", [0.75], 0.05)
IQR = Q3[0] - Q1[0]
lower_bound = Q1[0] - 1.5 * IQR
upper_bound = Q3[0] + 1.5 * IQR

df_combined_spark = df_combined_spark.filter((col('X_COORD_CD') >= lower_bound) & (col('X_COORD_CD') <= upper_bound))


In [43]:
# # Show the cleaned data
df_combined_spark.show()



+------------+----------+----------+---------+--------+--------------------+-----------------------+--------------+--------+--------------+-------------+-------+--------------+--------------------+--------------------+------------------+-------------------+----+-----+------------------+------------------+
|INCIDENT_KEY|OCCUR_DATE|OCCUR_TIME|     BORO|PRECINCT|       LOCATION_DESC|STATISTICAL_MURDER_FLAG|PERP_AGE_GROUP|PERP_SEX|     PERP_RACE|VIC_AGE_GROUP|VIC_SEX|      VIC_RACE|          X_COORD_CD|          Y_COORD_CD|          Latitude|          Longitude|YEAR|MONTH|X_COORD_CD_numeric|Y_COORD_CD_numeric|
+------------+----------+----------+---------+--------+--------------------+-----------------------+--------------+--------+--------------+-------------+-------+--------------+--------------------+--------------------+------------------+-------------------+----+-----+------------------+------------------+
|   146283083|2015-09-20|  04:00:00|    bronx|      40|                    |   

In [44]:
# Saving and exporting the dataframe to csv file
df_combined_spark = df_combined_spark.coalesce(1)



In [45]:
df_combined_spark.write.csv('final_shooting_incident_data_nypd.csv', header=True)


In [46]:
import sys
print(sys.executable)
print(sys.version)
print(sys.version_info)

C:\Users\macwa\anaconda3\envs\pyspark\python.exe
3.11.9 | packaged by Anaconda, Inc. | (main, Apr 19 2024, 16:40:41) [MSC v.1916 64 bit (AMD64)]
sys.version_info(major=3, minor=11, micro=9, releaselevel='final', serial=0)


In [ ]:
# Close the cursor and connection
spark.stop()
cursor.close()
conn.close()